In [ ]:
!pip uninstall -y bigframes
!pip install -U google-cloud-bigquery pandas pyarrow scikit-learn imbalanced-learn db-dtypes scipy joblib tqdm tqdm-joblib

In [ ]:
from google.colab import auth
auth.authenticate_user()

from google.cloud import bigquery
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from joblib import Parallel, delayed
import gc

PROJECT_ID = "comp90089assignment1uic"
# PROJECT_ID = "mimic-469105" # Vita's
client = bigquery.Client(project=PROJECT_ID)
print("✅ Authenticated and client initialized.")

✅ Authenticated and client initialized.


In [ ]:
%%bigquery icu_stays --project comp90089assignment1uic
# %%bigquery icu_stays --project mimic-469105
SELECT
  subject_id,
  hadm_id,
  stay_id,
  intime,
  outtime
FROM `physionet-data.mimiciv_3_1_icu.icustays`
WHERE intime IS NOT NULL

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
%%bigquery labs --project comp90089assignment1uic
# %%bigquery labs --project mimic-469105

SELECT
  le.subject_id,
  le.hadm_id,
  le.itemid,
  di.label AS lab_label,
  le.valuenum AS value,
  le.charttime
FROM `physionet-data.mimiciv_3_1_hosp.labevents` AS le
JOIN `physionet-data.mimiciv_3_1_hosp.d_labitems` AS di
  ON le.itemid = di.itemid
WHERE di.label IN ("Creatinine", "Urea nitrogen", "Sodium", "Potassium", "Chloride")
  AND le.valuenum IS NOT NULL

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
%%bigquery vitals --project comp90089assignment1uic
# %%bigquery vitals --project mimic-469105

SELECT
  stay_id,
  heart_rate_mean,
  sbp_mean,
  dbp_mean,
  mbp_mean,
  resp_rate_mean,
  temperature_mean,
  spo2_mean,
  glucose_mean
FROM `physionet-data.mimiciv_3_1_derived.first_day_vitalsign`


Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
%%bigquery icu_lab --project comp90089assignment1uic
# %%bigquery icu_lab --project mimic-469105

SELECT
  le.subject_id,
  le.hadm_id,
  icu.intime,
  icu.outtime,
  icu.stay_id,
  le.charttime,
  le.valuenum AS value,
  di.label AS lab_label
FROM `physionet-data.mimiciv_3_1_hosp.labevents` AS le
JOIN `physionet-data.mimiciv_3_1_hosp.d_labitems` AS di
  ON le.itemid = di.itemid
JOIN `physionet-data.mimiciv_3_1_icu.icustays` AS icu
  ON le.hadm_id = icu.hadm_id
  AND le.charttime BETWEEN icu.intime AND icu.outtime
WHERE di.label="Creatinine"  # IN ("Creatinine", "Urea nitrogen", "Sodium", "Potassium", "Chloride")
  AND le.valuenum IS NOT NULL

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
icu_lab["charttime"] = pd.to_datetime(icu_lab["charttime"])

# calculate the lab test time between admission time
icu_lab["hours_from_admit"] = (
    icu_lab["charttime"] - icu_lab["intime"]
).dt.total_seconds() / 3600

# only keep labs test in 24 hours after admit time
labs_first24h = icu_lab[icu_lab["hours_from_admit"].between(0, 24)]

# Merge ICU stays with first-day vitals using stay_id
df = labs_first24h.merge(vitals, on="stay_id", how="left")

# Optional: rename vitals columns for modeling clarity
df.rename(columns={
    'heart_rate_mean': 'heart_rate',
    'sbp_mean': 'sbp',
    'dbp_mean': 'dbp',
    'mbp_mean': 'meanbp',
    'resp_rate_mean': 'resp_rate',
    'temperature_mean': 'temperature',
    'spo2_mean': 'spo2',
    'glucose_mean': 'glucose',
    'value': 'lab_value'
}, inplace=True)


num_cols = ['heart_rate', 'sbp', 'dbp', 'meanbp', 'resp_rate', 'temperature', 'spo2', 'glucose', 'lab_value']
for col in num_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce").astype("float64")

print("✅ Final merged dataset shape:", df.shape)


✅ Final merged dataset shape: (184623, 17)


In [ ]:
def create_label(
    df,
    observation_end_hour=6,
    prediction_window_hours=12
):
    """
    Create AKI stage 2 labels WITHOUT data leakage.

    Parameters
    ----------
    df : pd.DataFrame
        Must contain ['stay_id', 'intime', 'charttime', 'lab_label', 'lab_value']
    observation_end_hour : int
        End of observation window (hours from admission)
    prediction_window_hours : int
        Length of prediction window (hours)

    Returns
    -------
    pd.DataFrame
        DataFrame with 'aki_label' column (1 = AKI stage 2, 0 = no AKI)
    """
    # Only work with creatinine measurements
    crea_df = df[df["lab_label"] == "Creatinine"].copy()
    crea_df.sort_values(["stay_id", "charttime"], inplace=True)

    # Calculate hours from admission for all records
    crea_df["hours_from_admit"] = (
        crea_df["charttime"] - crea_df["intime"]
    ).dt.total_seconds() / 3600

    aki_labels = []
    stays_processed = 0
    stays_excluded = 0

    for stay_id, group in crea_df.groupby("stay_id"):
        # ----- OBSERVATION WINDOW: [0, observation_end_hour) -----
        # This is the ONLY data we can use to establish baseline
        obs_window = group[
            (group["hours_from_admit"] >= 0) &
            (group["hours_from_admit"] < observation_end_hour)
        ]

        # Skip if no baseline data available
        if obs_window.empty:
            stays_excluded += 1
            continue

        # Use FIRST creatinine in observation window as baseline
        # This is the most conservative approach and mimics clinical practice
        baseline_creatinine = obs_window["lab_value"].iloc[0]

        # ----- PREDICTION WINDOW: [observation_end_hour, observation_end_hour + prediction_window_hours) -----
        prediction_window = group[
            (group["hours_from_admit"] >= observation_end_hour) &
            (group["hours_from_admit"] < observation_end_hour + prediction_window_hours)
        ]

        # Check for AKI Stage 2 in prediction window ONLY
        # KDIGO Stage 2: Creatinine ≥2x baseline OR ≥4.0 mg/dL
        aki_occurred = False
        if not prediction_window.empty:
            aki_events = prediction_window[
                (prediction_window["lab_value"] >= 2 * baseline_creatinine) |
                (prediction_window["lab_value"] >= 4.0)
            ]
            aki_occurred = not aki_events.empty

        # Label ALL records from this stay
        for idx in group.index:
            aki_labels.append({
                "index": idx,
                "stay_id": stay_id,
                "aki_label": int(aki_occurred),
                "baseline_creatinine": baseline_creatinine
            })

        stays_processed += 1

    # Create labels dataframe
    labels_df = pd.DataFrame(aki_labels)

    # Merge back to original dataframe
    result_df = df.merge(
        labels_df[["index", "aki_label", "baseline_creatinine"]],
        left_index=True,
        right_on="index",
        how="left"
    )

    # Remove stays without valid baseline
    result_df = result_df.dropna(subset=["aki_label"])

    print(f"[Info] Processed {stays_processed} stays")
    print(f"[Info] Excluded {stays_excluded} stays without baseline data")
    print(f"[Info] AKI prevalence: {result_df.groupby('stay_id')['aki_label'].max().mean():.2%}")

    return result_df


In [ ]:
# create label
df = create_label(df)
print(df.head())

from google.colab import files
df.to_csv("final_cohort.csv", index=False)
files.download('final_cohort.csv')

[Info] Processed 64423 stays
[Info] Excluded 25726 stays without baseline data
[Info] AKI prevalence: 5.13%
          subject_id   hadm_id              intime             outtime  \
29530.0     10048001  28426278 2175-02-05 02:02:00 2175-02-09 17:20:12   
121104.0    10433099  21832792 2148-10-04 16:43:37 2148-10-06 20:22:47   
48150.0     13675174  20934668 2164-07-03 09:32:29 2164-07-12 14:58:42   
122272.0    17937834  22464591 2140-06-27 03:00:00 2140-06-27 18:05:05   
51200.0     11171260  20920997 2156-05-22 11:13:17 2156-06-24 03:31:57   

           stay_id           charttime  lab_value   lab_label  \
29530.0   31975834 2175-02-05 03:36:00        3.4  Creatinine   
121104.0  38080894 2148-10-04 17:24:00        3.4  Creatinine   
48150.0   33221747 2164-07-04 00:00:00        3.4  Creatinine   
122272.0  38157201 2140-06-27 03:23:00        3.5  Creatinine   
51200.0   33426506 2156-05-22 15:02:00        3.6  Creatinine   

          hours_from_admit  heart_rate         sbp      